In [75]:
import utils
import Graph_Exploration
import importlib
importlib.reload(utils)
import networkx as nx


In [76]:
import sys
sys.prefix

'/home/deborah/FS24/social_media/project/project-git/Social-Media-Analytics---Project/.venv'

In [77]:
nodes,edges = utils.Load_data("data/musae_facebook_target.csv","data/musae_facebook_edges.csv")

In [109]:
G = utils.Create_Graph(nodes=nodes , edges=edges)

Louvain first step

In [110]:
def modularity_gain(node,neighbor):
    m = G.number_of_edges()
    di = G.degree(node)
    dj = G.degree(neighbor)
    if neighbor not in com_inv:
        return 1/(2*m)*(2-di*dj/m)
    else:
        com_index = com_inv[neighbor]
        members = com[com_index]
        shared_degree = 0
        for m in members:
            if m in G.neighbors(node):
                shared_degree +=2
        #print("shared_dergee",shared_degree)
        return 1/(2*m)*(shared_degree-di*dj/m)


#modularity_gain(0,18427)

com = {}
com_inv = {}
for v in G.nodes():
    #delete v from its community:
    if v in com_inv.keys():
        com_index = com_inv[v]
        del com_inv[v]
        com[com_index].remove(v) 
    #Find the neighbor with highest modularity gain
    #print("com_inv",com_inv)
    gains = []
    for neighbor in G.neighbors(v):
        gains.append((modularity_gain(v,neighbor),neighbor))
    closest_node = max(gains)[1]
    # Add v to the same community as the closest_node
    #check if closest node is already in a community:
    if closest_node in com_inv.keys():
        com_index = com_inv[closest_node]
        com_inv[v]=com_index
        com[com_index].add(v)
    else:
        #Add v and neighbor in a new created community
        com_index = len(com)
        com_inv[v]=com_index
        com_inv[closest_node]= com_index
        com[com_index] = {v,closest_node}
    
    if len(com)>1000:
        break
print(com)

{0: {0, 295, 1099, 396, 1365, 18427}, 1: {1, 10281}, 2: {2, 2629, 126}, 3: {3, 293, 1286, 713, 586, 1385, 9654, 1079}, 4: {14332, 4}, 5: {21768, 5}, 6: {17038, 6}, 7: {3305, 7}, 8: {544, 325, 358, 3975, 8}, 9: {864, 9, 1391, 2773, 983, 1215}, 10: {10, 1187, 14, 13511}, 11: {8106, 11}, 12: {19356, 12}, 13: {16282, 13}, 14: {353, 15}, 15: {16, 652, 10134}, 16: {17, 21059}, 17: {18, 3259}, 18: {9184, 19}, 19: {20}, 20: {5627, 21}, 21: {15868, 22}, 22: {7082, 23}, 23: {24, 6432}, 24: {344, 25}, 25: {26, 7139}, 26: {27, 21247}, 27: {11784, 28}, 28: {29, 20895}, 29: {17178, 30}, 30: {12542, 31}, 31: {32, 1024, 120, 2157}, 32: {33, 400, 241, 15191, 703}, 33: {34, 14106, 557, 407}, 34: {7898, 35}, 35: {18754, 36}, 36: {17363, 37}, 37: {14944, 38}, 38: {13130, 39}, 39: {768, 260, 40, 1231, 827, 1020, 19197}, 40: {41, 9484, 45}, 41: {1393, 42, 18030}, 42: {1273, 43, 188, 10997}, 43: {1126, 839, 44, 5165, 568}, 44: {1510, 12130, 46}, 45: {3409, 47}, 46: {48, 5541}, 47: {655, 49, 7734, 280, 1274},

In [111]:
print(len(com))
print(len(com_inv))
print(len(com.values()))

1001
2469
1001


In [136]:
#Second passage
import networkx as nx

def construct_community_graph(com, com_inv, G):
    # Initialize empty graph
    community_graph = nx.Graph()
    
    # Calculate total degree for each node in the original graph
    node_degrees = dict(G.degree())
    
    # Add nodes for each community and calculate total degrees
    total_degrees = {}
    for community, members in com.items():
        community_graph.add_node(community, size=len(members))
        total_degrees[community] = sum(node_degrees[node] for node in members)
    
    # Add edges between community nodes and calculate shared degrees
    for community1, members1 in com.items():
        for community2, members2 in com.items():
            if community1 != community2:
                shared_degree = sum(len(set(G.neighbors(node)).intersection(members2)) for node in members1)
                #print(shared_degree)
                if shared_degree > 0:
                    community_graph.add_edge(community1, community2, shared_degree=shared_degree)
        
    # Add total degree as node attribute
    nx.set_node_attributes(community_graph, total_degrees, 'total_degree')
    
    return community_graph



hyper_graph = construct_community_graph(com, com_inv,G)

In [137]:
print(hyper_graph.number_of_nodes())
print(hyper_graph.number_of_edges())


1001
792


In [ ]:
def modularity_gain(node,neighbor):
    m = G.number_of_edges()
    di = G.degree(node)
    dj = G.degree(neighbor)
    if neighbor not in com_inv:
        return 1/(2*m)*(2-di*dj/m)
    else:
        com_index = com_inv[neighbor]
        members = com[com_index]
        shared_degree = 0
        for m in members:
            if m in G.neighbors(node):
                shared_degree +=2
        #print("shared_dergee",shared_degree)
        return 1/(2*m)*(shared_degree-di*dj/m)


    #modularity_gain(0,18427)
def second_passage():
    com = {}
    com_inv = {}
    for v in G.nodes():
        #delete v from its community:
        if v in com_inv.keys():
            com_index = com_inv[v]
            del com_inv[v]
            com[com_index].remove(v) 
        #Find the neighbor with highest modularity gain
        #print("com_inv",com_inv)
        gains = []
        for neighbor in G.neighbors(v):
            gains.append((modularity_gain(v,neighbor),neighbor))
        closest_node = max(gains)[1]
        # Add v to the same community as the closest_node
        #check if closest node is already in a community:
        if closest_node in com_inv.keys():
            com_index = com_inv[closest_node]
            com_inv[v]=com_index
            com[com_index].add(v)
        else:
            #Add v and neighbor in a new created community
            com_index = len(com)
            com_inv[v]=com_index
            com_inv[closest_node]= com_index
            com[com_index] = {v,closest_node}
        
        if len(com)>1000:
            break
    print(com)

{0: {0, 295, 1099, 396, 1365, 18427}, 1: {1, 10281}, 2: {2, 2629, 126}, 3: {3, 293, 1286, 713, 586, 1385, 9654, 1079}, 4: {14332, 4}, 5: {21768, 5}, 6: {17038, 6}, 7: {3305, 7}, 8: {544, 325, 358, 3975, 8}, 9: {864, 9, 1391, 2773, 983, 1215}, 10: {10, 1187, 14, 13511}, 11: {8106, 11}, 12: {19356, 12}, 13: {16282, 13}, 14: {353, 15}, 15: {16, 652, 10134}, 16: {17, 21059}, 17: {18, 3259}, 18: {9184, 19}, 19: {20}, 20: {5627, 21}, 21: {15868, 22}, 22: {7082, 23}, 23: {24, 6432}, 24: {344, 25}, 25: {26, 7139}, 26: {27, 21247}, 27: {11784, 28}, 28: {29, 20895}, 29: {17178, 30}, 30: {12542, 31}, 31: {32, 1024, 120, 2157}, 32: {33, 400, 241, 15191, 703}, 33: {34, 14106, 557, 407}, 34: {7898, 35}, 35: {18754, 36}, 36: {17363, 37}, 37: {14944, 38}, 38: {13130, 39}, 39: {768, 260, 40, 1231, 827, 1020, 19197}, 40: {41, 9484, 45}, 41: {1393, 42, 18030}, 42: {1273, 43, 188, 10997}, 43: {1126, 839, 44, 5165, 568}, 44: {1510, 12130, 46}, 45: {3409, 47}, 46: {48, 5541}, 47: {655, 49, 7734, 280, 1274},

In [129]:
n1 = set(G.neighbors(0))
n2 = set(G.neighbors(295))
n1.intersection(n2)
n2
n2.intersection({0,18377})

{18377}

In [15]:
communities = nx.community.louvain_communities(G,seed=2)
print("There are",len(communities),"communities.")

There are 60 communities.


In [26]:
for comm_idx, community in enumerate(communities):
    for node_id in community:
        G.nodes[node_id]['louvain_communities'] = comm_idx

In [36]:
min_length, min_sublist = min((len(sublist), sublist) for sublist in communities)
print(min_length)
print(max(len(sublist) for sublist in communities))


8
3853


In [24]:
total_length = sum(len(sublist) for sublist in communities)

# Calculate the average length
average_length = total_length / len(communities)
average_length

374.5

In [31]:
G.nodes[100]

{'facebook_id': 2046450978914607,
 'page_name': 'Préfecture de police des Bouches-du-Rhône',
 'page_type': 'government',
 'louvain_communities': 9}

In [37]:
G,nodes = Graph_Exploration.Calculate_closeness_centrality(G,nodes)

In [50]:
#Louvain:
G
com = {}
com[0] = [51]
com[0].append(1)
com[20] = 2
print(51 in com.values())
len(com)
print(com)

SyntaxError: invalid syntax (4175900380.py, line 7)

In [70]:
def modularity_gain(node,neighbor):
    m = G.number_of_edges()
    di = G.degree(node)
    dj = G.degree(neighbor)
    if 'com' not in G.nodes[neighbor]:
        return 1/(2*m)*(2-di*dj/m)


modularity_gain(0,18427)

In [74]:
#com = {}
#_ = [G.nodes[node].pop('com', None) for node in G.nodes()]
for v in G.nodes():
    #delete v from its community:
    if 'com' in G.nodes[v]:
        com_index = G.nodes[v]['com']
        del G.nodes[v]['com']
        com[com_index].remove(v) 
    #Find the neighbor with highest modularity gain
    gains = []
    for neighbor in G[v]:
        gains.append((modularity_gain(v,neighbor),neighbor))
    closest_node = max(gains)[1]
    # Add v to the same community as the closest_node
    #check if closest node is already in a community:
    if 'com' in G.nodes[closest_node]:
        com_index = G.nodes[closest_node]['com']
        G.nodes[v]['com']=com_index
        com[com_index].append(v)
    else:
        #Add v and neighbor in a new created community
        com_index = len(com)
        G.nodes[v]['com']=com_index
        G.nodes[closest_node]['com'] = com_index
        com[com_index] = [v,closest_node]
    
    if len(com)>100:
        break
print(com)

TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [61]:
del G.nodes[18427]['com']
print(G.nodes[closest_node])

{'facebook_id': 1563883503866518, 'page_name': 'The Voice Global', 'page_type': 'tvshow'}
